In [ ]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [3]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=1)
search_results = search.invoke("Amrabat Fenerbahçe'ye transfer oldu mu?")
print(search_results)

tools = [search]


[{'url': 'https://www.cnnturk.com/spor/futbol/galeri/son-dakika-transfer-haberi-sofyan-amrabat-fenerbahce-icin-istanbula-ucuyor-iste-gelis-saati-2147458', 'content': "Ferdi transferinin ardından büyük bir gelir elde eden Fenerbahçe merkez orta saha transferinde mutlu sona ulaştı, 6 numara transferini bitirdi. Sarı-lacivertlilerin, Sofyan Amrabat'ın transferi için Serie A ekibi Fiorentina ile anlaşmaya vardı. Sofyan Amrabat 21 Ağustos 1996 yılında ..."}]


In [5]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [8]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

In [9]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [10]:
model_with_tools = model.bind_tools(tools)

In [11]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [12]:
response = model_with_tools.invoke([HumanMessage(content="Sofyan Amrabat Fenerbahçe'ye transfer oldu mu?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'Sofyan Amrabat Fenerbahçe transfer'}, 'id': 'call_mEnXtIl0zACr55jeVTpjZmmy', 'type': 'tool_call'}]


In [14]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(model, tools)

In [15]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='1ad57d9f-6c75-4d98-95f0-3ad4c28cef9c'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None}, id='run-6db81f07-b1ae-4e3b-b9f4-804f9f9c25ad-0', usage_metadata={'input_tokens': 81, 'output_tokens': 10, 'total_tokens': 91})]

In [16]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="Sofyan Amrabat Fenerbahçe'ye transfer oldu mu?")]}
)
response["messages"]

[HumanMessage(content="Sofyan Amrabat Fenerbahçe'ye transfer oldu mu?", id='4b8338b3-1d78-4ff2-85d6-e4853eeea1e2'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mMc1NncoXlxbYt1muzJ2vzuE', 'function': {'arguments': '{"query":"Sofyan Amrabat Fenerbahçe\'ye transfer oldu mu"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 95, 'total_tokens': 127}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6547e154-01b8-4062-bb2f-7e1c115de95e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': "Sofyan Amrabat Fenerbahçe'ye transfer oldu mu"}, 'id': 'call_mMc1NncoXlxbYt1muzJ2vzuE', 'type': 'tool_call'}], usage_metadata={'input_tokens': 95, 'output_tokens': 32, 'total_tokens': 127}),
 ToolMessage(content='[{"url": "https://beinsports.com.tr/haber/sofyan-amrabattan-g